In [ ]:
!pip install gym
!pip install tensorflow
!pip install -U "ray[rllib]"

In [ ]:
import gym, ray
import numpy as np
from ray.rllib.agents import dqn

- Create an env
- connect this env to the DQN agent (implemented in Ray)
- Train the agent for the env

In [ ]:
# |0|0|0|0|0|1|
#  X
#  ---------->
# negative reward for not reaching the terminal state -0.1 / step


# gym.Env: class defining a standard env


class SimpleCorridor(gym.Env):
  def __init__(self, config):
    # the position of an agent
    self.end_pos = 5 # term at cell 5
    self.cur_pos = 0 # init with cell 0

    # define action/state space
    self.action_space = gym.spaces.Discrete(2) # left/right -> 2 disc actions
    self.observation_space = gym.spaces.Discrete(self.end_pos + 1)
  
  def reset(self):
    self.cur_pos = 0 # init loc
    return self.cur_pos

  def step(self, action):
    # update the cur_pos based on the action
    # aciton Discrete(2) -> action 0 or action 1
    # right 0, left 1
    if action == 0 and self.cur_pos < self.end_pos:
      self.cur_pos += 1
    elif action == 1 and self.cur_pos > 0:
      self.cur_pos -= 1

    pseudo_info = {}
    if self.cur_pos >= self.end_pos:
      return 0, 1.0, True, pseudo_info
    else:
      return self.cur_pos, -0.1, False, pseudo_info

In [ ]:
# conenct this to ray DQN

ray.shutdown()
ray.init() # start ray training env

config = {
    "env": SimpleCorridor,
    "env_config": {
        "gamma": 0.8
    },
}

# prepare an agent
trainer = dqn.DQNTrainer(config=config)

# train the agent with 2 batches
for _ in range(5):
  print(trainer.train())

trainer.evaluate()